In [1]:
import sys
import ROOT as R
#R.EnableImplicitMT()
%jsroot on

In [2]:
R.gSystem.Load("/data/HPS/lib/libMoller.dylib")
Moller = R.Moller()
print("Version of Moller Class:",Moller.GetVersion())

Version of Moller Class: 0.2.0


In [3]:
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/rafo_minidst/hps_007966.?_Moller_4.2_minidst.root"
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/hps_007966.n_Moller_4.2.root"
#snapshot_file_name = "moller_rafo_filtered.root"
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/pass4kf/hps_007966.#_recon_5.2.1_minidst.root"
#snapshot_file_name = "moller_pass4kf_filtered.root"
data_files_pattern = "/data/HPS/data/physrun2016/Moller/Moller_MC/molv4I_HPS-PhysicsRun2016-Pass2_iss650_singles0_*_minidst.root"
ch_moller = R.TChain("MiniDST")
# nf=0
# for i in range(1,40):
#     nf += ch_moller.Add(data_files_pattern.replace("#",str(i)))
nf = ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
df = R.RDataFrame(ch_moller)
print(f"Added {nf} files to the chain, total of {n_events/1e6:7.2f}M events")

Added 504 files to the chain, total of    6.70M events


In [4]:
Particle_names=["Final state particle KF", "UC V0 vertex KF", "BSC V0 vertex KF", "TC V0 vertex KF", "UC Moller vertex KF",
"BSC Moller vertex KF", "TC Moller vertex KF", "Other electron KF", "UC VC vertex KF", "Final state particle GBL", "UC V0 vertex GBL",
"BSC V0 vertex GBL", "TC V0 vertex GBL", "UC Moller vertex", "BSC Moller vertex", "TC Moller vertex", "Other electron GBL",
"UC VC vertex GBL"]
All_Names = list(df.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):
        Store_Names.append(s)
        dat_type = df.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_mc_id             [RVec<int>           ] ecal_cluster_mc_pdg            [RVec<int>           ] 
ecal_cluster_mc_pdg_purity     [RVec<double>        ] ecal_cluster_nhits             [RVec<int>           ] 
ecal_cluster_seed_energy       [RVec<double>        ] ecal_cluster_seed_index        [RVec<int>           ] 
ecal_cluster_seed_ix           [RVec<int>           ] ecal_cluster_seed_iy           [RVec<int>           ] 
ecal_cluster_time              [RVec<double>        ] ecal_cluster_uncor_energy      [RVec<double>        ] 
ecal_cluster_uncor_hits        [RVec<vector<int>>   ] ecal_cluster_uncor_nhits       [RVec<int>           ] 
ecal_cluster_uncor_seed_energy [RVec<double>        ] ecal_cluster_uncor_seed_index  [RVec<int>           ] 
ecal_cluster_uncor_seed_ix     [RVec<int>           ] ecal_cluster_uncor_seed_iy     [RVec<int>           ] 
ecal_cluster_uncor_

In [5]:
dfx = Moller.Select_El_Pairs_MC(R.RDF.AsRNode(df),0., 2.,0.001,"mc_electron_pairs")
dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(dfx),0., 2.,"electron_pairs")
dfxx = Moller.Select_El_Pairs_MC(R.RDF.AsRNode(df),4., 100.,0.001, "mc_electron_pairs")
dfxx = Moller.Select_El_Pairs(R.RDF.AsRNode(dfxx),4., 100., "electron_pairs")

d = [dfx, dfxx]

for i in range(len(d)):
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "electron_pairs","part_","p4")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "mc_electron_pairs","mc_part_","mc_p4")
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 2.306*0.75, 2.306*1.15, "electron_pairs", "el_pairs_r1")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r1","part_","r1_p4");
    d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]), "el_pairs_r1", "el_pairs_r2")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r2","part_","r2_p4");
    d[i] = Moller.Refine_El_Pairs_3(R.RDF.AsRNode(d[i]), 0.01, "el_pairs_r2", "r2_", "el_pairs_r3")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r3","part_","r3_p4");

    # These cuts should not be done.
    d[i] = Moller.Refine_El_Pairs_X(R.RDF.AsRNode(d[i]), 0.040, 0.048, "r2_p4tht1", "r2_p4tht2" ,"el_pairs_r2", "el_pairs_rx")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_rx","part_","rx_p4");
    d[i] = d[i].Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")


dfx, dfxx = d

In [6]:
for n in dfx.GetColumnNames():
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):
        dat_type = dfx.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_mc_id             [RVec<int>           ] ecal_cluster_mc_pdg            [RVec<int>           ] 
ecal_cluster_mc_pdg_purity     [RVec<double>        ] ecal_cluster_nhits             [RVec<int>           ] 
ecal_cluster_seed_energy       [RVec<double>        ] ecal_cluster_seed_index        [RVec<int>           ] 
ecal_cluster_seed_ix           [RVec<int>           ] ecal_cluster_seed_iy           [RVec<int>           ] 
ecal_cluster_time              [RVec<double>        ] ecal_cluster_uncor_energy      [RVec<double>        ] 
ecal_cluster_uncor_hits        [RVec<vector<int>>   ] ecal_cluster_uncor_nhits       [RVec<int>           ] 
ecal_cluster_uncor_seed_energy [RVec<double>        ] ecal_cluster_uncor_seed_index  [RVec<int>           ] 
ecal_cluster_uncor_seed_ix     [RVec<int>           ] ecal_cluster_uncor_seed_iy     [RVec<int>           ] 
ecal_cluster_uncor_

In [7]:
 # Example - Check that the charge of electrons is -1, first with indirection, then with a direct selection on the part_pdg.
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<e_part_idx.size();++i){out.push_back(part_charge[e_part_idx[i]]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<part_charge.size();++i){if(part_pdg[i]==11) out.push_back(part_charge[i]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
h_n_part = dfx.Define("n_part","return part_pdg.size()").Histo1D(("h_n_part","Number of particles",11,-0.5,10.5),"n_part")
h_n_neg_track = dfx.Define("n_neg_track","int count=0;for(int i=0;i<track_omega.size();++i){if(track_omega[i]>0 && track_type[i]==1)count++;} return count;").Histo1D(("h_n_neg_track","Number of - tracks",11,-0.5,10.5),"n_neg_track")
h_n_electron = dfx.Define("n_electron","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Histo1D(("h_n_electron","Number of e-",11,-0.5,10.5),"n_electron")
h_n_moller = dfx.Define("n_moller","return electron_pairs.size()").Histo1D(("h_n_moller","Number of Moller pairs",11,-0.5,10.5),"n_moller")

h_track_time_diff = dfx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("h_track_time_diff","track_time_diff",500,-10.,10.),"track_time_diff")
hx_track_time_diff = dfxx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hx_track_time_diff","track_time_diff",500,-10.,10.),"track_time_diff")

h_track_chi2 = dfx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("h_track_chi2","track_chi2",500,0.,10.),"t_chi2")
hx_track_chi2 = dfxx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("xh_track_chi2","track_chi2",500,0.,10.),"t_chi2")

h_track_nhit = dfx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("h_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")
hx_track_nhit = dfxx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("hx_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")


In [8]:
%jsroot on
cc0 = R.TCanvas("cc0","cc0",1200,500)
cc0.Divide(2,2)
pad1 = cc0.cd(1)
pad1.SetLogy()
h_n_moller.SetTitle("Particle counts")
h_n_moller.SetStats(0)
h_n_moller.SetLineWidth(2)
h_n_moller.SetLineColor(R.kCyan)
h_n_moller.Draw("same")
h_n_electron.SetLineWidth(3)
h_n_electron.SetLineColor(R.kBlue)
h_n_electron.Draw("same")
h_n_neg_track.SetLineColor(R.kRed)
h_n_neg_track.Draw("same")
h_n_part.SetLineWidth(1)
h_n_part.SetLineColor(R.kGreen+2)
h_n_part.Draw("same")
legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_part.GetPtr(),"any particle","l")
legend.AddEntry(h_n_neg_track.GetPtr(),"neg. tracks","l")
legend.AddEntry(h_n_electron.GetPtr(),"electron","l")
legend.AddEntry(h_n_moller.GetPtr(),"Moller","l")
legend.Draw()
cc0.cd(2)
h_track_time_diff.SetLineWidth(1)
h_track_time_diff.SetLineColor(R.kGreen+2)
h_track_time_diff.Draw()
hx_track_time_diff.SetLineColor(R.kRed)
hx_track_time_diff.Draw("same")
cc0.cd(3)
h_track_chi2.SetLineWidth(1)
h_track_chi2.SetLineColor(R.kGreen+2)
h_track_chi2.Draw()
hx_track_chi2.SetLineColor(R.kRed)
hx_track_chi2.Draw("same")
cc0.cd(4)
h_track_nhit.SetStats(0)
h_track_nhit.SetLineColor(R.kRed)
h_track_nhit.Draw("same")
hx_track_nhit.SetStats(0)
hx_track_nhit.SetLineWidth(1)
hx_track_nhit.SetLineColor(R.kGreen+2)
hx_track_nhit.Draw("same")

cc0.Draw()

In [9]:
h_v0_type = dfx.Histo1D(("h_v0_type","V0 Type",17,-0.5,16.5),"v0_type")
#("h_n_v0","n_v0",11,-0.5,10.5)
cc1 = R.TCanvas("cc1","cc1",1000,500)
#cc1.Divide(1,2)
pad1= cc1.cd(1)
pad1.SetBottomMargin(0.2)
axis = h_v0_type.GetXaxis()
axis.SetLabelSize(0.035)
for i in range (0,16):
    axis.SetBinLabel(i+1,Particle_names[i])
h_v0_type.SetStats(0)
h_v0_type.SetFillColor(R.kBlue-10)
h_v0_type.Draw()
cc1.Draw()

In [10]:
h_v0_mass_UC = dfx.Define("v0_mass_UC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==13) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_UC","v0 mass UC; M_{inv} [GeV]",500,0.,0.2),"v0_mass_UC")
h_v0_mass_BSC = dfx.Define("v0_mass_BSC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==14) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_BSC","v0 mass UCM_{inv} [GeV]",500,0.,0.2),"v0_mass_BSC")
h_v0_mass_TC = dfx.Define("v0_mass_TC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==15) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_TC","v0 mass TCM_{inv} [GeV]",500,0.,0.2),"v0_mass_TC")

In [11]:
h_ee_mass_r2 = dfx.Histo1D(("h_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
hx_ee_mass_r2 = dfxx.Histo1D(("hx_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
h_mc_mass = dfx.Histo1D(("h_mc_mass","MC inv mass; M_{inv} [GeV]",500,0.,0.2),"mc_p4minv")
h_ee_mass_r3 = dfx.Histo1D(("h_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")
hx_ee_mass_r3 = dfxx.Histo1D(("hx_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")

h_ee_thtE1 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht1","p4E1")
h_ee_thth = dfx.Histo2D(("h_ee_thth","EE theta-theta;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"p4tht1","p4tht2")
h_mc_ee_thtE1 = dfx.Histo2D(("h_mc_ee_thtE1", "MC EE theta-E 1;#theta_{1} [rad];E_{1} [GeV]", 500, 0., 0.1, 500, 0., 2.5),
                           "mc_p4tht1", "mc_p4E1")
h_mc_ee_thth = dfx.Histo2D(("h_mc_ee_thth", "MC EE theta-theta;#theta_{1} [rad];#theta_{2} [rad]", 500, 0., 0.1, 500, 0., 0.1),
                          "mc_p4tht1", "mc_p4tht2")


In [12]:
cc2 = R.TCanvas("cc2","cc2",1200,1200)
cc2.Divide(2,2)
pad1= cc2.cd(1)
pad1.SetLogy()
h_ee_mass_r2.SetStats(0)
h_ee_mass_r2.SetLineColor(R.kOrange)
h_ee_mass_r2.SetLineWidth(2)
h_ee_mass_r2.Draw()
hx_ee_mass_r2.Scale(0.9)
hx_ee_mass_r2.SetStats(0)
hx_ee_mass_r2.SetLineColor(R.kRed)
hx_ee_mass_r2.SetLineWidth(2)
hx_ee_mass_r2.Draw("same")
h_mc_mass.SetLineColor(R.kGreen)
h_mc_mass.SetLineWidth(2)
h_mc_mass.Draw("same")
h_ee_mass_r3.SetStats(0)
h_ee_mass_r3.SetLineColor(R.kOrange+2)
h_ee_mass_r3.SetLineWidth(2)
h_ee_mass_r3.Draw("same")
h_ee_mass_r3.Fit("gaus")
# #hx_ee_mass.Scale(0.9)
# hx_ee_mass_r3.SetStats(0)
# hx_ee_mass_r3.SetLineColor(R.kRed+2)
# hx_ee_mass_r3.SetLineWidth(2)
# hx_ee_mass_r3.Draw("same")

# h_v0_mass_UC.SetStats(0)
# h_v0_mass_UC.SetLineWidth(1)
# h_v0_mass_UC.SetLineColor(R.kGreen+2)
# h_v0_mass_UC.Draw("same")
# h_v0_mass_BSC.SetLineWidth(1)
# h_v0_mass_BSC.SetLineColor(R.kCyan)
# h_v0_mass_BSC.Draw("same")
# h_v0_mass_TC.SetLineColor(R.kBlue)
# h_v0_mass_TC.Draw("same")

legend = R.TLegend(0.6,0.7,0.9,0.9)
legend.AddEntry(h_v0_mass_UC.GetPtr(),"Unconstrained Moller","l")
legend.AddEntry(h_v0_mass_BSC.GetPtr(),"Beamspot contrained Moller","l")
legend.AddEntry(h_v0_mass_TC.GetPtr(),"Target constrained Moller","l")
legend.AddEntry(h_mc_mass.GetPtr(),"Pre-Slic MC Moller","l")
legend.AddEntry(h_ee_mass_r2.GetPtr(),"Moller","l")
legend.AddEntry(hx_ee_mass_r2.GetPtr(),"Not Moller","l")
legend.Draw()

pad2 = cc2.cd(2)
pad2.SetLogz()
h_ee_thth.SetStats(0)
h_ee_thth.Draw("colz")
l1 = R.TLine(0.,0.04,0.04,0.)
l1.SetLineColor(R.kRed)
l1.SetLineWidth(2)
l1.Draw()
l2 = R.TLine(0.,0.048,0.048,0.)
l2.SetLineColor(R.kRed)
l2.SetLineWidth(2)
l2.Draw()
f1 = R.TF1("f1b","2*asin(0.000511/(2*2.3*sin(x/2)))",0.0044,0.1)
f1.SetLineColor(R.kGreen)
f1.Draw("same")

pad3 = cc2.cd(3)
pad3.SetLogz()
f2 = R.TF1("f2","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f2.SetLineColor(R.kRed)
h_ee_thtE1.SetStats(0)
h_ee_thtE1.Draw("colz")
f2.Draw("same")
pad4 = cc2.cd(4)
#f1 = R.TF1("f1","2*asin(2*0.11108696/sin(x/2))",0.0001,0.1)


cc2.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =        35442
NDf                       =          122
Edm                       =  1.26043e-06
NCalls                    =           84
Constant                  =      47667.7   +/-   80.9728     
Mean                      =    0.0485763   +/-   2.79515e-06 
Sigma                     =   0.00226036   +/-   2.67808e-06  	 (limited)


In [13]:
disp = dfx.Define("n_mc_el","int out=0;for(int i=0;i<mc_part_pdg.size(); ++i){if(mc_part_pdg[i]==11){out++;}}; return out").Display({"n_mc_el","mc_part_pdg","mc_part_z","mc_part_time"},50,20)
disp.Print()

+-----+------------+---------------+-------------+---------+
| Row | mc_part_z  | mc_part_time  | mc_part_pdg | n_mc_el | 
+-----+------------+---------------+-------------+---------+
| 10  | -4.295940  | 12000.000000  | 11          | 2       | 
|     | -4.295940  | 12000.000000  | 11          |         | 
+-----+------------+---------------+-------------+---------+
| 36  | -4.295940  | 41500.000000  | 11          | 2       | 
|     | -4.295940  | 41500.000000  | 11          |         | 
+-----+------------+---------------+-------------+---------+
| 37  | -4.295940  | 42000.000000  | 11          | 2       | 
|     | -4.295940  | 42000.000000  | 11          |         | 
+-----+------------+---------------+-------------+---------+
| 46  | -4.295940  | 51000.000000  | 11          | 2       | 
|     | -4.295939  | 51000.000000  | 11          |         | 
+-----+------------+---------------+-------------+---------+
| 57  | -4.295940  | 63000.000000  | 11          | 2       | 
|     | -4.295